To-do:
- bylo zaměstnáno počátkem roku 9780 dělníků

In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col("datum").dt.year() >= 1918
).drop_nulls(
    subset=["datum"]
).sort(
    by="datum"
)

In [4]:
import re

In [5]:
def vroce(retezec):
    roky = []
    fraze = [
        r'[Rr]ok \d{2,4}',
        r'[Vv] roce \d{2,4}',
        r'[Rr]oku \d{2,4}',
        r'[Rr]okem \d{2,4}'
    ]
    for f in fraze:
        nalezy = re.findall(f,retezec)
        pridej = [int(re.search(r'\d{2,4}', x).group()) for x in nalezy]
        pridej = [int(f'19{x}') if 18 <= x < 100 else int(f'20{x}') if x < 17 else x for x in pridej]
        roky += pridej
    return roky

In [6]:
test = 'V roce 2011 bylo dobře, zato v roce 2012 bylo ještě líp, ovšem roku 2013 se to začalo kazit a rokem 2014 dobré časy skončily. Rok 39 byl těžký, rok 89 veselý. Rok 14 byl rokem začátku války.'

In [7]:
vroce(test)

[1939, 1989, 2014, 2011, 2012, 2013, 2014]

In [8]:
test2 = 'Žádná zmínka'

In [9]:
vroce(test2)

[]

In [10]:
df = df.with_columns(pl.col("text").map_elements(vroce, return_dtype=pl.List(pl.Int32)).alias("roky"))

In [11]:
vyvoj = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').median()
)

In [12]:
alt.Chart(vyvoj.to_pandas(),width=800).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [13]:
vyvoj2 = df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').quantile(0.95)
)

In [14]:
alt.Chart(vyvoj2.to_pandas(),width=800).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [15]:
alt.Chart(
    df.drop_nulls(
    subset=["datum"]
).explode(
    "roky"
).with_columns(
    pl.col("roky") - pl.col("datum").dt.year()
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col('roky').max()
).to_pandas(),
    width=800
).mark_bar().encode(alt.X('datum:T'),alt.Y('roky:Q'))

alt.Chart(...)

In [35]:
df.explode("roky").drop_nulls(subset=['roky']).sort(by="roky",descending=True)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,roky
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,i32
"""Posl. Toužil""","""1920ns_ps_stenprot_240schuz_s240002.htm""",null,"""Slavná sněmovno! Poslanecká sněmovna zabývá se v pravidelných lhůtách otázkou nezaměstnanosti. Podnět takového pravidelného projednávání dokazuje, jak…",5,"""ns""",1920,240,2,1923-12-11,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925""","""240. schůze""",true,true,9780
"""Zpravodaj posl. kol. Remeš pravil v rozpočtovém výboru""","""1929ns_ps_stenprot_242schuz_s242001.htm""",null,"""Jsou jen dvě možnosti, buď šetřiti nebo tisknouti. O první víme všichni ze zkušenosti, že to jest účinný prostředek, o druhé této zkušenosti nemáme. R…",4,"""ns""",1929,242,1,1933-02-02,"""NS RČS 1929-1935""","""242. schůze""",true,true,9600
"""Místopředseda vlády a ministr práce a sociálních věcí ČR Petr Nečas""","""2006se_stenprot_013schuz_s013001.htm""",null,"""Vážený pane předsedo, vážené paní senátorky, vážení páni senátoři, dámy a pánové. Návrh zákona, který byl schválen Poslaneckou sněmovnou podle § 90 zá…",6,"""se""",2006,13,1,2008-04-23,"""PČR, Senát 2006-2008""","""13. schůze""",true,true,9111
"""Zpravodaj posl. inž. Černý""","""1925ns_ps_stenprot_021schuz_s021004.htm""",null,"""Slavná sněmovno! Zákon navržený, který je velmi jasný, dal jedné politické straně příležitost, aby zase vyvolala extempore, které jsou, řekněme docela…",2,"""ns""",1925,21,4,1926-05-18,"""Národní shromáždění československé / Poslanecká sněmovna1925 – 1929""","""21. schůze""",true,true,8760
"""Sen. Jarolim (německy)""","""1920ns_se_stenprot_132schuz_s132001.htm""",null,"""Slavný senáte! Program vlády, který pan ministerský předseda předložil slavnému senátu, nemůže nás, německé sociální demokraty, v žádném směru uspokoj…",11,"""ns""",1920,132,1,1922-11-09,"""NS RČS 1920-1925""","""132. schůze""",true,true,8297
null,"""1954ns_stenprot_036schuz_s036010.htm""",null,"""Národné výbory nesú rovnako veľkú zodpovednosť za rozvoj platených služieb pri zavádzaní normatívov hmotnej zainteresovanosti a novej mzdovej sústavy,…",1,"""ns""",1954,36,10,1960-02-17,"""NS RČS 1954-1960""","""36. schůze""",true,true,7442
"""Sen. Beutel (německy)""","""1925ns_se_stenprot_145schuz_s145011.htm""",null,"""Slavný senáte! Dříve, nežli budu probírati sociální pojištění, musím se zabývati prohlášením, které pan ministr veřejných prací podal k velikému neště…",3,"""ns""",1925,145,11,1928-10-11,"""NS RČS 1925-1929""","""145. schůze""",true,true,7200
"""Zpravodaj posl. Bergmann""","""1929ns_ps_stenprot_311schuz_s311005.htm""",null,"""Slavná sněmovno! Posl. Beran, Pik, Zeminová, Košek, Schäfer, Ježek, dr Hodina a druzi podali iniciativní návrh, aby srážky, které se ústavním činitelů…",4,"""ns""",1929,311,5,1933-12-19,"""NS RČS 1929-1935""","""311. schůze""",true,true,7200
"""Posl. Najman""","""1920ns_ps_stenprot_178schuz_s178004.htm""",null,"""Slavná sněmovno! Je charakteristické, když jedná se o konsumní spolky, že senát vychází vstříc těmto spolkům a že snižuje stupnici daně výdělkové. Byl…",12,"""ns""",1920,178,4,1922-12-12,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925""","""178. schůze""",true,true,7195


In [39]:
df.explode("roky").drop_nulls(subset=['roky']).sort(by="roky",descending=True).head(1).select(pl.col("text")).item()

'Slavná sněmovno! Poslanecká sněmovna zabývá se v pravidelných lhůtách otázkou nezaměstnanosti. Podnět takového pravidelného projednávání dokazuje, jak malorysá jest politika dnešní vlády. Otázka nezaměstnanosti jest tak důležitá, že sahá přímo na kořeny státu. Za velikou chybu pokládám, když ministerstvo soc. péče je toho mínění, že nezaměstnanost jest zjevem přechodným a že jedině také proto prodlužuje zákon o nezaměstnanosti pouze na dobu 6 měsíců. Komunistická strana hned na počátku projednávání nezaměstnanosti varovala před takovým optimismem a volala po důkladném řešení této otázky. Také v naléhavé interpelaci z 5. února t. r. varovali jsme vládu před podceňováním nezaměstnanosti a žádali jsme o nejrychlejší a důsledné její potírání. Nezaměstnanost stává se dnes ohromným a sotva ovládatelným sociálním problémem, který hrozí svou vlastní podstatou a svým zjevem, že nebude lze obyčejnými prostředky tento problém odstraniti. Avšak zodpovědní činitelé zůstali při svém původním názoru